In [1]:
import os 
import geopandas as gpd
import xarray as xr
import netCDF4 as nc
import pandas as pd
os.chdir("C:/Users/cshat/eo/EFFIS-ERA5-EUROPE")


In [2]:
fg_0 = "data/era5/processed/E-OBS/fg_0.nc"
hu_0 = "data/era5/processed/E-OBS/hu_0.nc"
rr_0 = "data/era5/processed/E-OBS/rr_0.nc"
tx_0 = "data/era5/raw/ERA5_EOBS_01Deg_1995-2010/tx_ens_mean_0.1deg_reg_1995-2010_v30.0e.nc"
fg_1 = "data/era5/processed/E-OBS/fg_1.nc"
hu_1 = "data/era5/processed/E-OBS/hu_1.nc"
rr_1 = "data/era5/processed/E-OBS/rr_1.nc"
tx_1 = "data/era5/raw/ERA5_EOBS_01Deg_2011-2024/tx_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc"
fg_0 = xr.open_dataset(fg_0)
hu_0 = xr.open_dataset(hu_0)
rr_0 = xr.open_dataset(rr_0)
tx_0 = xr.open_dataset(tx_0)
fg_1 = xr.open_dataset(fg_1)
hu_1 = xr.open_dataset(hu_1)
rr_1 = xr.open_dataset(rr_1)
tx_1 = xr.open_dataset(tx_1)

effis = r"C:\Users\cshat\eo\Forest-Fires-Europe\data\processed\Fire_Data\EFFIS\effis-filtered.shp"
effis = gpd.read_file(effis)

In [11]:
tx_0 = tx_0.sel(time=slice('2000-01-01', None))
tx_1 = tx_1.sel(time=slice('2000-01-01', None))
tx_0 = tx_0.rename({'longitude': 'x', 'latitude': 'y'})
tx_1 = tx_1.rename({'longitude': 'x', 'latitude': 'y'})

In [10]:
fg_1

<xarray.Dataset> Size: 10GB
Dimensions:      (x: 646, y: 386, time: 4930)
Coordinates:
  * x            (x) float64 5kB -24.35 -24.25 -24.15 ... 39.95 40.05 40.15
  * y            (y) float64 3kB 71.15 71.05 70.95 70.85 ... 32.85 32.75 32.65
  * time         (time) datetime64[ns] 39kB 2011-01-01 2011-01-02 ... 2024-06-30
Data variables:
    spatial_ref  int32 4B ...
    fg           (time, y, x) float64 10GB ...

In [ ]:
tx = xr.merge([tx_0, tx_1])
fg = xr.merge([fg_0, fg_1])
hu = xr.merge([hu_0, hu_1])
rr = xr.merge([rr_0, rr_1])



In [12]:
import geopandas as gpd
import xarray as xr
import pandas as pd

# Assume effis is your GeoPandas DataFrame with polygons
# Assume fg, tg, rr, hu are your xarray Datasets

# Set the CRS of effis to match ERA5 (EPSG:4326)
effis = effis.to_crs(epsg=4326)

# Compute centroids
effis['centroid'] = effis.geometry.centroid

# Extract x (longitude) and y (latitude) from the centroid column
effis['x'] = effis['centroid'].x  # longitude
effis['y'] = effis['centroid'].y  # latitude

# Filter for the first 10 points
effis_subset = effis.iloc[:10]

# Define datasets with variable names
datasets = {
    'fg': fg_0,  # Variable 'fg' in Dataset fg
    'tx': tx_0,  # Variable 'tg' in Dataset tg
    'rr': rr_0,  # Variable 'rr' in Dataset rr
    'hu': hu_0,   # Variable 'hu' in Dataset hu
    'fg': fg_1,  # Variable 'fg' in Dataset fg
    'tx': tx_1,  # Variable 'tg' in Dataset tg
    'rr': rr_1,  # Variable 'rr' in Dataset rr
    'hu': hu_1   # Variable 'hu' in Dataset hu
}

# Dictionary to store extracted data for all variables
extracted_data = {}

# Loop over the first 10 points in effis_subset
for idx, row in effis_subset.iterrows():
    x = row['x']  # longitude
    y = row['y']  # latitude
    
    # Extract data for this centroid from each dataset
    point_data = {}
    for var_name, ds in datasets.items():
        point_data[var_name] = ds[var_name].sel(x=x, y=y, method='nearest').to_dataframe()
    
    # Combine data for this point across variables
    combined_point_data = pd.concat(point_data.values(), axis=1)
    extracted_data[idx] = combined_point_data

# Combine all points into a single DataFrame
result = pd.concat(extracted_data, axis=0, names=['point_idx', 'time'])

# Reset index if needed
result = result.reset_index()

# Print the result
print(result)

# Save to CSV
result.to_csv('data/processed/effis/extracted_climate_data.csv', index=False)

C:\Users\cshat\AppData\Local\Temp\ipykernel_16592\69681605.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  effis['centroid'] = effis.geometry.centroid


       point_idx       time      x      y    fg          y         x  \
0              0 2011-01-01  27.75  36.15  3.46  36.149861  27.74986   
1              0 2011-01-02  27.75  36.15  3.45  36.149861  27.74986   
2              0 2011-01-03  27.75  36.15  2.86  36.149861  27.74986   
3              0 2011-01-04  27.75  36.15  2.51  36.149861  27.74986   
4              0 2011-01-05  27.75  36.15  3.80  36.149861  27.74986   
...          ...        ...    ...    ...   ...        ...       ...   
49295          9 2024-06-26  23.95  40.85  2.41  40.849861  23.94986   
49296          9 2024-06-27  23.95  40.85  2.43  40.849861  23.94986   
49297          9 2024-06-28  23.95  40.85  2.53  40.849861  23.94986   
49298          9 2024-06-29  23.95  40.85  2.46  40.849861  23.94986   
49299          9 2024-06-30  23.95  40.85  2.87  40.849861  23.94986   

              tx         x         y    rr         x          y         hu  
0      13.500000  27.74986  36.14986   0.0  27.74986  36.1

In [15]:
result


,point_idx,time,x,y,fg,y,x,tx,x,y,rr,x,y,hu
0,0,2011-01-01,27.75,36.15,3.46,36.149861,27.74986,13.500000,27.74986,36.14986,0.0,27.74986,36.149861,68.407997
1,0,2011-01-02,27.75,36.15,3.45,36.149861,27.74986,14.420000,27.74986,36.14986,0.0,27.74986,36.149861,73.989502
2,0,2011-01-03,27.75,36.15,2.86,36.149861,27.74986,14.589999,27.74986,36.14986,21.8,27.74986,36.149861,84.968948
3,0,2011-01-04,27.75,36.15,2.51,36.149861,27.74986,13.420000,27.74986,36.14986,28.8,27.74986,36.149861,70.630501
4,0,2011-01-05,27.75,36.15,3.80,36.149861,27.74986,13.190000,27.74986,36.14986,0.0,27.74986,36.149861,81.995499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49295,9,2024-06-26,23.95,40.85,2.41,40.849861,23.94986,30.740000,23.94986,40.84986,0.0,23.94986,40.849861,NaN
49296,9,2024-06-27,23.95,40.85,2.43,40.849861,23.94986,29.730000,23.94986,40.84986,0.0,23.94986,40.849861,NaN
49297,9,2024-06-28,23.95,40.85,2.53,40.849861,23.94986,28.680000,23.94986,40.84986,0.0,23.94986,40.849861,NaN
49298,9,2024-06-29,23.95,40.85,2.46,40.849861,23.94986,29.980000,23.94986,40.84986,0.0,23.94986,40.849861,NaN
